In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd /content/drive/My Drive/SA/Code/spoof_detection_deep_features

/content/drive/My Drive/SA/Code/spoof_detection_deep_features


In [5]:
!pip install librosa


  Using cached librosa-0.7.2.tar.gz (1.6 MB)
  Using cached audioread-2.1.8.tar.gz (21 kB)
  Using cached resampy-0.2.2.tar.gz (323 kB)
     |████████████████████████████████| 3.7 MB 489 kB/s eta 0:00:01
  Using cached SoundFile-0.10.3.post1-py2.py3-none-any.whl (21 kB)
     |████████████████████████████████| 20.2 MB 126 kB/s  eta 0:00:01
     |████████████████████████████████| 399 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 12.1 MB/s eta 0:00:01
  Created wheel for librosa: filename=librosa-0.7.2-py3-none-any.whl size=1612883 sha256=932ee4a00b1dd0b8d08cf7820cbc471375d3fe809de67ca30581e3cf4ecfec22
  Stored in directory: /home/rohita/.cache/pip/wheels/4b/72/78/82101d52cdfc32755f1606294a72d380aabb3fe70e105a71a0
  Created wheel for audioread: filename=audioread-2.1.8-py3-none-any.whl size=23091 sha256=29d61d9a3fde15b5aefed5d44a683e06853cf28a006ba1979214be4ceccd738b
  Stored in directory: /home/rohita/.cache/pip/wheels/fb/86/30/67eca0131420c96216f3a8a63a69795f6f

In [1]:
import keras 
import numpy as np
import librosa as lb
import pandas as pd
import os
import sys
# import pandas as pd
from keras.utils import to_categorical
# import soundfile as sf
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model,to_categorical
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
# import speechpy as sp
# import statistics
from keras import backend as K
from keras.layers import Dense, Activation, Flatten

Using TensorFlow backend.
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rohita/rohit/spoof/work3/l

## This is for train data

In [2]:
filename = "/home/rohita/rohit/spoof/ASVspoof2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt"

# open the file for reading
filehandle = open(filename, 'r')
train_protocol = []
while True:
    # read a single line
    line = (filehandle.readline())
    train_protocol.append(line)
    if not line:
        break

# close the pointer to that file
filehandle.close()

train_protocol = [s[:-1] for s in train_protocol]

train_protocol = pd.DataFrame([s.split(' ') for s in train_protocol])

train_protocol.columns = ['speaker_id','file_id', 'blah','system_id', 'label']

train_protocol = train_protocol[['speaker_id', 'file_id', 'system_id', 'label']]

train_protocol = train_protocol.dropna()

train_protocol.drop_duplicates(subset ="file_id",keep = 'first',inplace = True)

# This step was done to create a dataframe to keep track of the speaker id and system id of the samples but due to lack of space i deleted the flac files so i cant do it now

In [5]:
#import names of files in dataset
path = r'/home/rohita/rohit/spoof/ASVspoof2019/LA/ASVspoof2019_LA_train/flac'
files = []
missing=[]
print(path)
for r, d, f in os.walk(path):
    for file in f:
        if '.flac' in file  :        
            files.append(os.path.join(r, file))
        else:
            missing.append(file)
print(len(files))


files = [s.split('/') for s in files]

files = [s[-1] for s in files]

files = [s[:-5] for s in files]

files1 = [s.split(' ') for s in files]

for s in files:
    if len(s)>12:
        print(s)
        files.remove(s)
print(len(files))

train_file_id = list(train_protocol.iloc[:,1])

/home/rohita/rohit/spoof/ASVspoof2019/LA/ASVspoof2019_LA_train/flac
25380


In [7]:
train_labels = []
train_audio = []
for count,audio in enumerate(files):
    index = train_file_id.index(audio)
    if bool(index) == True:
        train_audio.append(lb.load('/home/rohita/rohit/spoof/ASVspoof2019/LA/ASVspoof2019_LA_train/flac/'+audio+'.flac',sr=16000))
        train_labels.append(train_protocol.iloc[index,3])
    if count%100 == 0 :
        print(count)
        
print(len(train_audio))
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train.npy",np.array(train_audio))
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_labels.npy",train_labels)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [8]:
train_protocol_ordered_based_on_files = pd.DataFrame(columns=['speaker_id','file_id','system_id','label'])
for count,audio in enumerate(files):
    index = train_file_id.index(audio)
    if bool(index) == True:
        train_protocol_ordered_based_on_files=train_protocol_ordered_based_on_files.append(train_protocol.iloc[index])
    if count%100 == 0 :
        print(count)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [9]:
train_protocol_ordered_based_on_files = train_protocol_ordered_based_on_files.reset_index(drop=True)

In [10]:
unique, counts = np.unique(train_protocol_ordered_based_on_files.iloc[:,2], return_counts=True)
print(dict(zip(unique, counts)))

{'A01': 3800, 'A06': 3800, 'A02': 3800, '-': 2579, 'A03': 3800, 'A05': 3800, 'A04': 3800}


In [11]:
index = train_file_id.index('LA_T_4941896')
print(train_protocol.iloc[index])

speaker_id         LA_0087
file_id       LA_T_4941896
system_id              A01
label                spoof
Name: 4170, dtype: object


In [12]:
for count, i in enumerate(train_protocol_ordered_based_on_files.iloc[:,-2]):
    if i =='-':
        train_protocol_ordered_based_on_files.iloc[count,-2] = 0
    elif i == 'A01':
        train_protocol_ordered_based_on_files.iloc[count,-2] = 1
    elif i == 'A02':
        train_protocol_ordered_based_on_files.iloc[count,-2] = 2
    elif i == 'A03':
        train_protocol_ordered_based_on_files.iloc[count,-2] = 3
    elif i == 'A04':
        train_protocol_ordered_based_on_files.iloc[count,-2] = 4
    elif i == 'A05':
        train_protocol_ordered_based_on_files.iloc[count,-2] = 5
    elif i == 'A06':
        train_protocol_ordered_based_on_files.iloc[count,-2] = 6

In [ ]:
train_audio.shape

## load data for splitting the train part of ASVspoof 2019 dataset in train and val

In [ ]:
labels = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_labels.npy")

labels=list(labels)

labels1 = []
for i in labels:
    if i == 'bonafide':
        labels1.append(1)
    else:
        labels1.append(0)

## Trim the audio to median length and set human to 1 and spoof to 0

In [13]:
import numpy as np
import pandas as pd
import pickle

audio = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train.npy",allow_pickle=True)
labels = train_protocol_ordered_based_on_files.iloc[:,-1]
labels=list(labels)

labels1 = []
for i in labels:
    if i == 'bonafide':
        labels1.append(0)
    else:
        labels1.append(1)

In [ ]:
audio = [s[0] for s in audio]

audio = np.array(audio)

audio_shapes = np.array([s.shape[0] for s in audio])

np.median(audio_shapes)

trimmed_audio = pad_sequences(audio,maxlen=51222,value=0.0,padding='pre',truncating='post',dtype='float32')

trimmed_audio[2]

np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train.npy",trimmed_audio)

## shuffle only the samples and labels

In [ ]:
labels1

In [7]:
unique, counts = np.unique(labels1, return_counts=True)
print(dict(zip(unique, counts)))

dataset = pd.DataFrame(audio)
dataset['label'] = labels1
print(dataset.head(10))


shuffled_df = dataset.sample(frac=1,random_state=4)

# Put all the human class in a separate dataset.
human_samples = shuffled_df.loc[shuffled_df['label'] == 0]
human_samples = human_samples.append([human_samples]*8,ignore_index=True)
print(human_samples.shape)
#Randomly select 3674 observations from the spoofed (majority class)
spoofed_samples = shuffled_df.loc[shuffled_df['label'] == 0].sample(frac=1,random_state=42)

# Concatenate both dataframes again
normalized_df = pd.concat([human_samples, spoofed_samples])

normalized_df = normalized_df.sample(frac=1,random_state=42)

print(normalized_df.shape)


{0: 22800, 1: 2579}
          0         1         2         3         4         5         6  \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000031  0.000031  0.000031  0.000031  0.000031  0.000031  0.000031   
5 -0.000488 -0.000488 -0.000336 -0.000183 -0.000092 -0.000031  0.000214   
6 -0.001434 -0.000946 -0.000397 -0.000214 -0.000214 -0.000153  0.000671   
7  0.001434  0.000916  0.000549  0.000549  0.000488  0.000061  0.000275   
8 -0.002319 -0.002380 -0.002502 -0.002777 -0.002747 -0.002502 -0.002777   
9  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7         8         9  ...     51213     51214     51215     51216  \
0  0.000000  0.000000  0.000000  ... -0.000061 -0.000061 -0.000031 -0.0001

In [8]:
X_train = np.array(normalized_df.iloc[:32208,:51221])
y_train = np.array(normalized_df.iloc[:32208,-1])

X_val = np.array(normalized_df.iloc[32208:,:51221])
y_val = np.array(normalized_df.iloc[32208:,-1])

np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train.npy",X_train)
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_train_labels.npy",y_train)
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_val.npy",X_val)
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_val_labels.npy",y_val)

## shuffle and make proper dataset

In [14]:
unique, counts = np.unique(train_protocol_ordered_based_on_files.iloc[:,3], return_counts=True)
print(dict(zip(unique, counts)))

dataset = pd.DataFrame(audio)
dataset['speaker_id'] = train_protocol_ordered_based_on_files['speaker_id']
dataset['file_id'] = train_protocol_ordered_based_on_files['file_id']
dataset['system_id'] = train_protocol_ordered_based_on_files['system_id']
dataset['label'] = [0 if s == 'bonafide' else 1 for s in train_protocol_ordered_based_on_files.iloc[:,-1]]

# print(dataset.head(10))


shuffled_df = dataset.sample(frac=1,random_state=4)

# Put all the human class in a separate dataset.
human_samples = shuffled_df.loc[shuffled_df['system_id'] == 0]
# human_samples = human_samples.append(human_samples,ignore_index=True)
# human_samples = human_samples.sample(frac=1,random_state =4)

#Randomly select 3674 observations from the spoofed (majority class)
spoofed_samples = shuffled_df.loc[shuffled_df['system_id'] == 2].sample(frac=1,random_state=42)

# Concatenate both dataframes again
normalized_df = pd.concat([human_samples, spoofed_samples])
del train_protocol_ordered_based_on_files
del audio
del dataset
del shuffled_df
del human_samples
del spoofed_samples
normalized_df = normalized_df.sample(frac=1,random_state=42)
# normalized_df = normalized_df.drop_duplicates(keep='first',inplace = True)
# print(normalized_df.shape)

# normalized_df['audio'] = normalized_df[normalized_df.columns[0:51222]].values.tolist()
# # print(normalized_df.head(10))
# normalized_df.drop(normalized_df.iloc[:, 0:51222], inplace = True, axis = 1) 
# # print(normalized_df.head(10))

{'spoof': 22800, 'bonafide': 2579}


In [15]:
# normalized_df.to_pickle("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_df.pkl")
X_train = np.array(normalized_df.iloc[:,:-4])
y_train = np.array(normalized_df.iloc[:,-2])
normalized_df = normalized_df.iloc[:,-4:]
normalized_df.to_pickle("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_df_sub.pkl")

# X_val = np.array(normalized_df.iloc[17766:,:51221])
# y_val = np.array(normalized_df.iloc[17766:,-2])


np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_sub.npy",X_train)
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_sub_labels.npy",y_train)
# np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_val_sid.npy",X_val)
# np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_train_val_sid_labels.npy",y_val)

In [16]:
# for count,i in enumerate(labels1):
#     print(i,train_protocol_ordered_based_on_files.iloc[count,-1])
len(X_train)

25379

## Dev set

In [2]:
filename = "/home/rohita/rohit/spoof/ASVspoof2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt"

# open the file for reading
filehandle = open(filename, 'r')
dev_protocol = []
while True:
    # read a single line
    line = (filehandle.readline())
    dev_protocol.append(line)
    if not line:
        break

# close the pointer to that file
filehandle.close()

In [3]:
dev_protocol = [s[:-1] for s in dev_protocol]

dev_protocol = pd.DataFrame([s.split(' ') for s in dev_protocol])

dev_protocol.columns = ['speaker_id','file_id', 'blah','system_id', 'label']

dev_protocol = dev_protocol[['speaker_id', 'file_id', 'system_id', 'label']]

dev_protocol = dev_protocol.dropna()

In [4]:
#import names of files in dataset
path = r'/home/rohita/rohit/spoof/ASVspoof2019/LA/ASVspoof2019_LA_dev/flac'
files = []
missing=[]
print(path)
for r, d, f in os.walk(path):
    for file in f:
        if '.flac' in file  :        
            files.append(os.path.join(r, file))
        else:
            missing.append(file)
print(len(files))


files = [s.split('/') for s in files]

files = [s[-1] for s in files]

files = [s[:-5] for s in files]

files1 = [s.split(' ') for s in files]

for s in files:
    if len(s)>12:
        print(s)
        files.remove(s)

dev_file_id = list(dev_protocol.iloc[:,1])

/home/rohita/rohit/spoof/ASVspoof2019/LA/ASVspoof2019_LA_dev/flac
24986
LA_D_A2940120
LA_D_A8082083
LA_D_A9165322
LA_D_A9063912
LA_D_A6763914
LA_D_A8486194
LA_D_A5086574
LA_D_A7944567
LA_D_A2004586
LA_D_A2486210
LA_D_A1045602
LA_D_A8631378
LA_D_A9163645
LA_D_A5637003
LA_D_A2300370
LA_D_A1640883
LA_D_A2113518
LA_D_A5452066
LA_D_A2667946
LA_D_A8555382
LA_D_A4397042
LA_D_A9282295
LA_D_A7526873
LA_D_A5781599
LA_D_A4372259
LA_D_A7495873
LA_D_A9442421
LA_D_A5058238
LA_D_A4011660
LA_D_A9738230
LA_D_A8630807
LA_D_A1600475
LA_D_A6997796
LA_D_A6934713
LA_D_A2091569
LA_D_A9509678
LA_D_A5053798
LA_D_A1146342
LA_D_A7770316
LA_D_A9567385
LA_D_A5088604
LA_D_A6866194
LA_D_A9832811
LA_D_A8202532
LA_D_A7729162
LA_D_A4464508
LA_D_A1981362
LA_D_A1305336
LA_D_A3079793
LA_D_A3322786
LA_D_A9497856
LA_D_A4129936
LA_D_A7005427
LA_D_A2936391
LA_D_A4645563
LA_D_A8203727
LA_D_A6583126
LA_D_A5688215
LA_D_A3276106
LA_D_A3333968
LA_D_A9087858
LA_D_A9474465
LA_D_A8551139
LA_D_A9939030
LA_D_A8133373
LA_D_A6270180
LA_D

In [5]:
dev_labels = []
dev_audio = []
for count,audio in enumerate(files):
    index = dev_file_id.index(audio)
    if bool(index) == True:
        dev_audio.append(lb.load('/home/rohita/rohit/spoof/ASVspoof2019/LA/ASVspoof2019_LA_dev/flac/'+audio+'.flac',sr=16000))
        dev_labels.append(dev_protocol.iloc[index,3])
    if count%100 == 0 :
        print(count)
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev.npy",np.array(dev_audio))
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_labels.npy",dev_labels)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

## if you want to run this the dev data is already prepared 

In [11]:
dev_protocol_ordered_based_on_files = pd.DataFrame(columns=['speaker_id','file_id','system_id','label'])
for count,audio in enumerate(files):
    index = dev_file_id.index(audio)
    if bool(index) == True:
        dev_protocol_ordered_based_on_files=dev_protocol_ordered_based_on_files.append(dev_protocol.iloc[index])
    if count%100 == 0 :
        print(count)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [12]:
dev_protocol_ordered_based_on_files = dev_protocol_ordered_based_on_files.reset_index(drop=True)

In [13]:
dev_protocol_ordered_based_on_files['label'] = [0 if s == 'bonafide' else 1 for s in dev_protocol_ordered_based_on_files.iloc[:,-1]]

In [14]:
dev_protocol_ordered_based_on_files.to_pickle("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_df.pkl")

In [21]:
import numpy as np
import pandas as pd
import pickle

audio = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev.npy", allow_pickle=True)
# labels = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_labels.npy")
# labels=list(labels)

# labels1 = []
# for i in labels:
#     if i == 'bonafide':
#         labels1.append(1)
#     else:
#         labels1.append(0)

In [12]:
audio = [s[0] for s in audio]

audio = np.array(audio)

audio_shapes = np.array([s.shape[0] for s in audio])

np.median(audio_shapes)

trimmed_audio = pad_sequences(audio,maxlen=51222,value=0.0,padding='pre',truncating='post',dtype='float32')

trimmed_audio[2]

np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev.npy",trimmed_audio)

In [9]:
unique, counts = np.unique(dev_protocol_ordered_based_on_files.iloc[:,3], return_counts=True)
print(dict(zip(unique, counts)))

{'spoof': 22296, 'bonafide': 2547}


In [10]:
unique, counts = np.unique(dev_protocol_ordered_based_on_files.iloc[:,2], return_counts=True)
print(dict(zip(unique, counts)))

{'A03': 3716, 'A05': 3716, '-': 2547, 'A02': 3716, 'A01': 3716, 'A06': 3716, 'A04': 3716}


In [8]:
for count, i in enumerate(dev_protocol_ordered_based_on_files.iloc[:,-2]):
    if i =='-':
        dev_protocol_ordered_based_on_files.iloc[count,-2] = 0
    elif i == 'A01':
        dev_protocol_ordered_based_on_files.iloc[count,-2] = 1
    elif i == 'A02':
        dev_protocol_ordered_based_on_files.iloc[count,-2] = 2
    elif i == 'A03':
        dev_protocol_ordered_based_on_files.iloc[count,-2] = 3
    elif i == 'A04':
        dev_protocol_ordered_based_on_files.iloc[count,-2] = 4
    elif i == 'A05':
        dev_protocol_ordered_based_on_files.iloc[count,-2] = 5
    elif i == 'A06':
        dev_protocol_ordered_based_on_files.iloc[count,-2] = 6

In [23]:
# unique, counts = np.unique(dev_protocol_ordered_based_on_files.iloc[:,3], return_counts=True)
# print(dict(zip(unique, counts)))

dataset = pd.DataFrame(audio)
dataset['speaker_id'] = dev_protocol_ordered_based_on_files['speaker_id']
dataset['file_id'] = dev_protocol_ordered_based_on_files['file_id']
dataset['system_id'] = dev_protocol_ordered_based_on_files['system_id']
dataset['label'] = [0 if s == 'bonafide' else 1 for s in dev_protocol_ordered_based_on_files.iloc[:,-1]]

print(dataset.head(10))


shuffled_df = dataset.sample(frac=1,random_state=4)

# Put all the human class in a separate dataset.
human_samples = shuffled_df.loc[shuffled_df['system_id'] == 0]
# human_samples = human_samples.append(human_samples,ignore_index=True)
# human_samples = human_samples.sample(frac=1,random_state =4)

#Randomly select 3674 observations from the spoofed (majority class)
spoofed_samples = shuffled_df.loc[shuffled_df['system_id'] == 2].sample(frac=1,random_state=42)

# Concatenate both dataframes again
normalized_df = pd.concat([human_samples, spoofed_samples])
del dev_protocol_ordered_based_on_files
del audio
del dataset
del shuffled_df
del human_samples
del spoofed_samples
normalized_df = normalized_df.sample(frac=1,random_state=42)

X_train = np.array(normalized_df.iloc[:,:-4])
y_train = np.array(normalized_df.iloc[:,-2])
normalized_df = normalized_df.iloc[:,-4:]
normalized_df.to_pickle("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_df_sub.pkl")


np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_sub.npy",X_train)
np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_sub_labels.npy",y_train)
# np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_val.npy",X_val)
# np.save("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_val_labels.npy",y_val)

                                                   0      1 speaker_id  \
0  [-0.00091552734, -0.0023498535, -0.0043945312,...  16000    LA_0071   
1  [3.0517578e-05, 3.0517578e-05, 3.0517578e-05, ...  16000    LA_0071   
2  [3.0517578e-05, 3.0517578e-05, 3.0517578e-05, ...  16000    LA_0073   
3  [-0.0022277832, -0.002166748, -0.0022277832, -...  16000    LA_0074   
4  [-0.0032348633, -0.0030822754, -0.0033569336, ...  16000    LA_0072   
5  [-0.0039978027, -0.0036621094, -0.0034484863, ...  16000    LA_0077   
6  [-0.00021362305, 0.00018310547, 0.0019226074, ...  16000    LA_0070   
7  [6.1035156e-05, 6.1035156e-05, 6.1035156e-05, ...  16000    LA_0075   
8  [-0.00064086914, -0.001373291, -0.0024719238, ...  16000    LA_0102   
9  [0.0023498535, 0.002319336, 0.0026245117, 0.00...  16000    LA_0072   

        file_id system_id  label  
0  LA_D_9616032         5      1  
1  LA_D_4264656         2      1  
2  LA_D_6273410         4      1  
3  LA_D_6947992         5      1  
4  LA_D_94